In [2]:
from peewee import *

db = SqliteDatabase('people.db')

class Person(Model):
    name = CharField()
    birthday = DateField()

    class Meta:
        """Model metadata is “anything that’s not a field”, 
        such as ordering options (ordering), 
        database table name (db_table), 
        or human-readable singular and plural names (verbose_name and verbose_name_plural). 
        None are required, and adding class Meta to a model is completely optional."""
        database = db # This model uses the "people.db" database.
        
        
class Pet(Model):
    owner = ForeignKeyField(Person, backref='pets')
    name = CharField()
    animal_type = CharField()

    class Meta:
        database = db # this model uses the "people.db" database

In [3]:
db.connect()


True

In [4]:
db.create_tables([Person, Pet])

In [6]:
from datetime import date
uncle_bob = Person(name='Bob', birthday=date(1960, 1, 15))
uncle_bob.save() # bob is now stored in the database

1

In [7]:
grandma = Person.create(name='Grandma', birthday=date(1935, 3, 1))
herb = Person.create(name='Herb', birthday=date(1950, 5, 5))

In [8]:
grandma.name = 'Grandma L.'
grandma.save()  # Update grandma's name in the database.

1

In [9]:
bob_kitty = Pet.create(owner=uncle_bob, name='Kitty', animal_type='cat')
herb_fido = Pet.create(owner=herb, name='Fido', animal_type='dog')
herb_mittens = Pet.create(owner=herb, name='Mittens', animal_type='cat')
herb_mittens_jr = Pet.create(owner=herb, name='Mittens Jr', animal_type='cat')

In [10]:
herb_mittens.delete_instance()

1

In [11]:
herb_fido.owner = uncle_bob
herb_fido.save()

1

In [16]:
grandma = Person.select().where(Person.name == 'Grandma L.').get()
print(grandma.name)

Grandma L.


In [17]:
for person in Person.select():
    print(person.name)

Bob
Bob
Grandma L.
Herb


In [18]:
query = Pet.select().where(Pet.animal_type == 'cat')
for pet in query:
    print(pet.name, pet.owner.name)

Kitty Bob
Mittens Jr Herb


In [19]:
for pet in Pet.select().join(Person).where(Person.name == 'Bob'):
    print(pet.name)

Kitty
Fido


In [20]:
for pet in Pet.select().where(Pet.owner == uncle_bob).order_by(Pet.name):
    print(pet.name)

Fido
Kitty


In [21]:
for person in Person.select().order_by(Person.birthday.desc()):
    print(person.name, person.birthday)

Bob 1960-01-15
Bob 1960-01-15
Herb 1950-05-05
Grandma L. 1935-03-01


In [22]:
d1940 = date(1940, 1, 1)
d1960 = date(1960, 1, 1)
query = (Person
         .select()
         .where((Person.birthday < d1940) | (Person.birthday > d1960)))

for person in query:
    print(person.name, person.birthday)

Bob 1960-01-15
Bob 1960-01-15
Grandma L. 1935-03-01


In [23]:
query = (Person
         .select()
         .where(Person.birthday.between(d1940, d1960)))

for person in query:
    print(person.name, person.birthday)

Herb 1950-05-05


In [24]:
for person in Person.select():
    print(person.name, person.pets.count(), 'pets')

Bob 0 pets
Bob 2 pets
Grandma L. 0 pets
Herb 1 pets


In [25]:
query = (Person
         .select(Person, Pet)
         .join(Pet, JOIN.LEFT_OUTER)
         .order_by(Person.name, Pet.name))
for person in query:
    # We need to check if they have a pet instance attached, since not all
    # people have pets.
    if hasattr(person, 'pet'):
        print(person.name, person.pet.name)
    else:
        print(person.name, 'no pets')


Bob no pets
Bob Fido
Bob Kitty
Grandma L. no pets
Herb Mittens Jr


In [26]:
query = Person.select().order_by(Person.name).prefetch(Pet)
for person in query:
    print(person.name)
    for pet in person.pets:
        print('  *', pet.name)

Bob
Bob
  * Kitty
  * Fido
Grandma L.
Herb
  * Mittens Jr


In [27]:
db.close()

True